# ECE324: Assignment 2, Part I

## Hyperparameters

- Activation function $\in$ {sigmoid, ReLU, Linear}.
- Learning rate `alpha`.
- Number of `epochs`.
- Random seed `rng`.

## Steps

- [x] Import `traindata.csv`, `trainlabel.csv`, `validdata.csv`, and `validlabel.csv` via `np.loadtxt()`.
- [x] Code the neural network function `Neuron` that takes in {input $I$}.
    - [x] Initialize weights to random values between 0 and 1.
    - [x] Internal parameters: `alpha`, `activation function` (callable object with two functions: `activate` (default __call__) and `grad` to produce gradient), `weights`, and `bias`.
    - [x] `grad_desc` funciton that takes in a full epoch of data and computes average $\frac{\partial \text{loss}}{\partial w_i}$ for each $w_i$ and bias $b$ -> adjusts parameters using $\alpha$.
- [x] Code a `mean_squared_error` function that takes in neuron output `Y` and compares it with true label `L` and calculates $(I-L)^2$.
- [x] Meta-function to run a training trial with the specified hyperparameters {$\alpha$, `epochs`, `activation`, `random seed`} and output the following values for each epoch:
    - [x] Training loss.
    - [x] Validation loss.
    - [x] Training accuracy.
    - [x] Validation accuracy.
- [x] Function to 'properly plot' the T+V loss vs. epoch on one graph.
- [x] Function to 'properly plot' the T+V accuracy vs. epoch on one graph.


In [78]:
##############
# IMPORT BOX #
##############

import numpy as np
import matplotlib.pyplot as plt 

In [79]:
###################
# DATA IMPORT BOX #
###################

train_X = np.loadtxt('traindata.csv', delimiter=',')
train_Y = np.loadtxt('trainlabel.csv', delimiter=',')
valid_X = np.loadtxt('validdata.csv', delimiter=',')
valid_Y = np.loadtxt('validlabel.csv', delimiter=',')

print("=== Validating Data ===")
print("Dimensions of training X: \t{}".format(train_X.shape))
print("Dimensions of training Y: \t{}".format(train_Y.shape))
print("")
print("Dimensions of validation X: \t{}".format(valid_X.shape))
print("Dimensions of validation Y: \t{}".format(valid_Y.shape))


=== Validating Data ===
Dimensions of training X: 	(200, 9)
Dimensions of training Y: 	(200,)

Dimensions of validation X: 	(20, 9)
Dimensions of validation Y: 	(20,)


In [84]:
class Neuron(object):
    def __init__(self, num_inputs, activ_func, rand_seed=0, alpha=0.01):
        self.rand_seed = rand_seed
        np.random.seed(rand_seed)
        self.weights = np.random.random(num_inputs)
        self.bias = np.random.random()
        self.activ_func = activ_func
        self.alpha = alpha

    def Z(self, inp):
        return inp.dot(self.weights) + self.bias
    
    def __call__(self, inp):
        return self.activ_func(inp.dot(self.weights) + self.bias)

    def train(self, X_train, Y_train, X_valid, Y_valid, num_epochs):
        train_loss_v_epoch = []
        valid_loss_v_epoch = []

        train_accuracy_v_epoch = []
        valid_accuracy_v_epoch = []


        for e in range(num_epochs):
            train_loss = self.grad_desc(X_train, Y_train)
            train_loss_v_epoch.append(train_loss)

            valid_loss = self.mean_loss(X_valid, Y_valid)
            valid_loss_v_epoch.append(valid_loss)

            train_acc = self.get_accuracy(X_train, Y_train)
            train_accuracy_v_epoch.append(train_acc)

            valid_acc = self.get_accuracy(X_valid, Y_valid)
            valid_accuracy_v_epoch.append(valid_acc)

        return train_loss_v_epoch, valid_loss_v_epoch, train_accuracy_v_epoch, valid_accuracy_v_epoch

    
    def grad_desc(self, X, Y):
        """
        Function to perform one step of gradient descent on an epoch of data.

        1. Calculate $\frac{\partial \text{loss} }{\partial \text{parameter} }$ for each parameter w_i and b.
            a. d loss_j / d Y[j]
            b. d Y[j] / d Z[j] where Z[j] is the unactivated output of the neuron.
            c. d Z[j] / d w_i = X[j][i]; d Z[j] / d b = 1

            Multiply a*b*c to get d loss_j / d param_i
        2. Calculate the average gradient for each parameter.
        3. Apply the update rule to each parameter.
        4. Return (average_loss, training_accuracy, validation_accuracy)
        """

        # Instantiating Gradients, Loss
        weight_grads = np.zeros(self.weights.size)
        bias_grad = 0
        avg_loss = 0

        # Iterating through each training example to calculate gradients for parameters (and avg loss)
        for j in range(len(Y)):
            pred = self(X[j])
            loss = (pred - Y[j])**2

            # Solving for components to put into chain rule
            dldy = 2*(pred - Y[j])
            dydz = self.activ_func.grad(self.Z(X[j]))
            dzdw = np.copy(X[j])

            # Updating averages via back propagation (chain rule)
            weight_grads += (dldy*dydz*dzdw)/len(Y)
            bias_grad += (dldy*dydz*1)/len(Y)
            avg_loss += loss/len(Y)

        # Updating parameters based on gradients and learning rate

        self.weights -= weight_grads*self.alpha
        self.bias -= bias_grad*self.alpha

        return avg_loss

    def mean_loss(self, X, Y):
        pred = self(X)
        sq_diff = (pred - Y)**2
        return np.mean(sq_diff)
    
    def get_accuracy(self, X, Y):
        """
        Function to get the accuracy of the model for predicting values of Y based on X.
        """
        one_if_diff = np.abs(np.round(self(X)) - Y)
        num_incorrect = sum(one_if_diff)
        accuracy = (len(Y)-num_incorrect)/len(Y)
        return accuracy

In [101]:
class linear_activation(object):
    def __init__(self):
        self.name = 'Linear'
        return
    
    def __call__(self, inp):
        """
        Input is a scalar. We apply a linear activation function by returning the initial value.
        """

        return inp 
    
    def grad(self, inp):
        """
        Linear activation function has $y = x$ -> $dy/dx = 1$
        """
        return 1

class sigmoid_activation(object):
    def __init__(self):
        self.name = 'Sigmoid'
        return
    
    def __call__(self, inp):
        """
        Input is a scalar. We apply a linear activation function by returning the initial value.
        """

        return 1/(1+np.exp(-inp)) 
    
    def grad(self, inp):
        """
        Linear activation function has $y = x$ -> $dy/dx = 1$
        """
        return self(inp)*(1-self(inp))
        
class ReLU_activation(object):
    def __init__(self):
        self.name = 'ReLU'
        return
    
    def __call__(self, inp):
        """
        Input is a scalar. We apply a linear activation function by returning the initial value.
        """

        return max(0, inp)
    
    def grad(self, inp):
        """
        Linear activation function has $y = x$ -> $dy/dx = 1$
        """
        return np.heaviside(inp, 1)
        

linear = linear_activation()
sigmoid = sigmoid_activation()
ReLU = ReLU_activation()

In [102]:
def plot_report(train_loss_v_epoch, valid_loss_v_epoch, train_accuracy_v_epoch, valid_accuracy_v_epoch):
    plt.plot(train_loss_v_epoch)
    plt.plot(valid_loss_v_epoch)
    plt.title('Training and Validation Loss Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Mean Loss')
    plt.legend(['train', 'validation'])
    plt.show()

    plt.plot(train_accuracy_v_epoch)
    plt.plot(valid_accuracy_v_epoch)
    plt.title('Training and Validation Accuracy Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['train', 'validation'])
    plt.show()

In [104]:
################
# TESTING CELL #
################
def get_report(activation_func, num_epochs=200, alpha=0.01, rand_seed=0, plot=True):
    NN = Neuron(len(train_X[0]), activation_func, alpha=alpha, rand_seed=rand_seed)

    train_loss_v_epoch, valid_loss_v_epoch, train_accuracy_v_epoch, valid_accuracy_v_epoch = NN.train(train_X, train_Y, valid_X, valid_Y, num_epochs)

    print("Activation Function: \t{}".format(NN.activ_func.name))
    print("Learning Rate: \t\t{}".format(NN.alpha))
    print("Number of Epochs: \t{}".format(num_epochs))
    print("Random Seed: \t\t{}".format(NN.rand_seed))

    print('=================================')
    print("Final Training Accuracy: \t{}".format(train_accuracy_v_epoch[-1]))
    print("Final Validation Accuracy: \t{}".format(valid_accuracy_v_epoch[-1]))
    print("Final Training Loss (avg): \t{}".format(train_loss_v_epoch[-1]))
    print("Final Validation Loss (avg): \t{}".format(valid_loss_v_epoch[-1]))

    if plot:
        plot_report(train_loss_v_epoch, valid_loss_v_epoch, train_accuracy_v_epoch, valid_accuracy_v_epoch)

    # TODO: Return a dict with the results from the trial.

In [105]:
"""
linear = linear_activation()
sigmoid = sigmoid_activation()
ReLU = ReLU_activation()
"""
get_report(activation_func, num_epochs=200, alpha=0.01, rand_seed=0, plot=True):

